# Restaurant Recommender System Based On GPS

Sistem rekomendasi ini merupakan sistem rekomendasi restoran yang menggunakan alogritma K-Nearest Neighbour. Dalam alogirtma yang ada matrix yang digunakan adalah dengan menggunakan Location yang berupa Latitude dan Longitude serta jenis dari restoran. K-Neighbour yang digunakan adalah 5, untuk mengambil 5 index teratas yang mirip.

# Instalasi
Sebelum menginstall, dimohon untuk menjalankan command pada command prompt

`pip install numpy`

`pip install pandas`

`pip install fuzzywuzzy`

`pip install sklearn`

## Code

pertama-tama dilakukan import numpy,pandas,sklearn.neighbor, dan fuzzywuzzy. numpy dan pandas digunakan untuk pemorsesan data, sklearn.neigbors merupakan model dari KNN yang akan digunakan dan fuzzywuzzy digunakan untuk string matching yang akan digunakan dalam pencarian data.


In [1]:
import numpy as np
import pandas as pd
from fuzzywuzzy import process
from sklearn.neighbors import NearestNeighbors
from geopy import distance
import random
pd.__version__

C:\Users\Andodo\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


'1.1.4'

In [2]:
restaurant_id = pd.read_csv("datasets/geoplaces2.csv",usecols=["placeID",'longitude','latitude'],dtype={"placeID":"int32","longitude":"float32","latitude":"float32"}).sort_values(by="placeID").reset_index(drop=True)
restaurant_id

,placeID,latitude,longitude
0,132560,23.752304,-99.166916
1,132561,23.726818,-99.126503
2,132564,23.730925,-99.145187
3,132572,22.141647,-100.992714
4,132583,18.922291,-99.234329
...,...,...,...
125,135088,18.876011,-99.219887
126,135104,23.752981,-99.168434
127,135106,22.149710,-100.976089
128,135108,22.136253,-100.933586


In [3]:
restaurant_record = pd.read_csv("datasets/chefmozcuisine.csv",usecols=["placeID","Rcuisine"],dtype={"placeID":"int32","Rcuisine":"str"}).sort_values(by="placeID")
restaurant_record = restaurant_record[restaurant_record.placeID.isin(restaurant_id.placeID)].reset_index(drop=True)
restaurant_record = restaurant_record.merge(restaurant_id,on="placeID")
restaurant_record


,placeID,Rcuisine,latitude,longitude
0,132560,Regional,23.752304,-99.166916
1,132572,Cafeteria,22.141647,-100.992714
2,132583,Fast_Food,18.922291,-99.234329
3,132584,Mexican,23.752365,-99.165291
4,132594,Mexican,23.752167,-99.165710
...,...,...,...,...
106,135086,Burgers,22.141420,-101.013954
107,135088,Cafeteria,18.876011,-99.219887
108,135104,Mexican,23.752981,-99.168434
109,135106,Mexican,22.149710,-100.976089


In [4]:
restaurant_features = pd.concat([restaurant_record["Rcuisine"].str.get_dummies(sep=",")]).set_index(restaurant_record["placeID"])

restaurant_features_all = restaurant_features.reset_index()
restaurant_features_all

,placeID,American,Armenian,Bakery,Bar,Bar_Pub_Brewery,Breakfast-Brunch,Burgers,Cafe-Coffee_Shop,Cafeteria,...,Fast_Food,International,Italian,Japanese,Mediterranean,Mexican,Pizzeria,Regional,Seafood,Vietnamese
0,132560,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,132572,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,132583,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,132584,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,132594,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,135086,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
107,135088,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
108,135104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
109,135106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
restaurant_features = pd.concat([restaurant_record["Rcuisine"].str.get_dummies(sep=",")]).set_index(restaurant_record["placeID"])

restaurant_features_all = restaurant_features.reset_index()
restaurant_features_all


,placeID,American,Armenian,Bakery,Bar,Bar_Pub_Brewery,Breakfast-Brunch,Burgers,Cafe-Coffee_Shop,Cafeteria,...,Fast_Food,International,Italian,Japanese,Mediterranean,Mexican,Pizzeria,Regional,Seafood,Vietnamese
0,132560,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,132572,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,132583,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,132584,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,132594,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,135086,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
107,135088,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
108,135104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
109,135106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
restaurant_feature_gps = restaurant_features_all.merge(restaurant_id,on="placeID")
restaurant_feature_gps

,placeID,American,Armenian,Bakery,Bar,Bar_Pub_Brewery,Breakfast-Brunch,Burgers,Cafe-Coffee_Shop,Cafeteria,...,Italian,Japanese,Mediterranean,Mexican,Pizzeria,Regional,Seafood,Vietnamese,latitude,longitude
0,132560,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,23.752304,-99.166916
1,132572,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,22.141647,-100.992714
2,132583,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,18.922291,-99.234329
3,132584,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,23.752365,-99.165291
4,132594,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,23.752167,-99.165710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,135086,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,22.141420,-101.013954
107,135088,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,18.876011,-99.219887
108,135104,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,23.752981,-99.168434
109,135106,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,22.149710,-100.976089


In [7]:
restaurant_full_record = pd.read_csv("datasets/geoplaces2.csv",usecols=["placeID","name","address"],dtype={"placeID":"int32","name":"str","address":"str","price":"str"},encoding='utf_8').sort_values(by="placeID").reset_index(drop=True)
restaurant_full_record = restaurant_full_record.merge(restaurant_record,on="placeID")
restaurant_full_record

,placeID,name,address,Rcuisine,latitude,longitude
0,132560,puesto de gorditas,frente al tecnologico,Regional,23.752304,-99.166916
1,132572,Cafe Chaires,?,Cafeteria,22.141647,-100.992714
2,132583,McDonalds Centro,Rayon sn col. Centro,Fast_Food,18.922291,-99.234329
3,132584,Gorditas Dona Tota,?,Mexican,23.752365,-99.165291
4,132594,tacos de barbacoa enfrente del Tec,?,Mexican,23.752167,-99.165710
...,...,...,...,...,...,...
106,135086,Mcdonalds Parque Tangamanga,Lateral Salvador Nava Martinez 3145,Burgers,22.141420,-101.013954
107,135088,Cafeteria cenidet,Interior Internado Palmira SN,Cafeteria,18.876011,-99.219887
108,135104,vips,?,Mexican,23.752981,-99.168434
109,135106,El Rincon de San Francisco,Universidad 169,Mexican,22.149710,-100.976089


In [8]:
restaurant_rating_record = pd.read_csv("datasets/rating_final.csv",usecols=["placeID","rating","userID"],dtype={"placeID":"int32","rating":"int32","userID":"str"})
restaurant_rating_record = restaurant_rating_record[restaurant_rating_record.placeID.isin(restaurant_id.placeID)]
restaurant_rating_df = restaurant_rating_record.groupby(by=["placeID"]).mean().reset_index()
restaurant_rating_df = restaurant_rating_df[restaurant_rating_df.placeID.isin(restaurant_record.placeID)]
restaurant_rating_df


,placeID,rating
0,132560,0.500000
3,132572,1.000000
4,132583,1.000000
5,132584,1.333333
6,132594,0.600000
...,...,...
124,135086,0.800000
125,135088,1.000000
126,135104,0.857143
127,135106,1.200000


In [9]:
restaurant_df_gps = restaurant_rating_df.merge(restaurant_id,on="placeID")
restaurant_df_gps

,placeID,rating,latitude,longitude
0,132560,0.500000,23.752304,-99.166916
1,132572,1.000000,22.141647,-100.992714
2,132583,1.000000,18.922291,-99.234329
3,132584,1.333333,23.752365,-99.165291
4,132594,0.600000,23.752167,-99.165710
...,...,...,...,...
90,135086,0.800000,22.141420,-101.013954
91,135088,1.000000,18.876011,-99.219887
92,135104,0.857143,23.752981,-99.168434
93,135106,1.200000,22.149710,-100.976089


In [10]:
restaurant_full_record = restaurant_full_record.merge(restaurant_rating_df,on="placeID")
restaurant_full_record

,placeID,name,address,Rcuisine,latitude,longitude,rating
0,132560,puesto de gorditas,frente al tecnologico,Regional,23.752304,-99.166916,0.500000
1,132572,Cafe Chaires,?,Cafeteria,22.141647,-100.992714,1.000000
2,132583,McDonalds Centro,Rayon sn col. Centro,Fast_Food,18.922291,-99.234329,1.000000
3,132584,Gorditas Dona Tota,?,Mexican,23.752365,-99.165291,1.333333
4,132594,tacos de barbacoa enfrente del Tec,?,Mexican,23.752167,-99.165710,0.600000
...,...,...,...,...,...,...,...
106,135086,Mcdonalds Parque Tangamanga,Lateral Salvador Nava Martinez 3145,Burgers,22.141420,-101.013954,0.800000
107,135088,Cafeteria cenidet,Interior Internado Palmira SN,Cafeteria,18.876011,-99.219887,1.000000
108,135104,vips,?,Mexican,23.752981,-99.168434,0.857143
109,135106,El Rincon de San Francisco,Universidad 169,Mexican,22.149710,-100.976089,1.200000


In [11]:
restaurant_df = restaurant_features.merge(restaurant_df_gps,on="placeID").sort_values(by="placeID").reset_index(drop=True)
restaurant_df = restaurant_df.drop(['placeID','rating'], axis=1)
restaurant_df

,American,Armenian,Bakery,Bar,Bar_Pub_Brewery,Breakfast-Brunch,Burgers,Cafe-Coffee_Shop,Cafeteria,Chinese,...,Italian,Japanese,Mediterranean,Mexican,Pizzeria,Regional,Seafood,Vietnamese,latitude,longitude
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,23.752304,-99.166916
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,22.141647,-100.992714
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,18.922291,-99.234329
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,23.752365,-99.165291
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,23.752167,-99.165710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,22.141420,-101.013954
107,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,18.876011,-99.219887
108,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,23.752981,-99.168434
109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,22.149710,-100.976089


In [12]:
model_cnn =  NearestNeighbors(algorithm='auto', n_neighbors=10)
model_cnn.fit(restaurant_df)
distances, indices = model_cnn.kneighbors(restaurant_df,n_neighbors = 10)

In [73]:
def get_index_from_type(coords,name=None):
    index = random.sample(range(1,111), 20)
    if(name):
        restaurant_result = restaurant_full_record[restaurant_full_record["Rcuisine"]==name]
    else:
        restaurant_result = restaurant_full_record.loc[index]
    resto_distance = []
    for i in restaurant_result.index.tolist():
        resto_distance.append(distance.distance((restaurant_result.latitude[i],restaurant_result.longitude[i]),coords).km)
    restaurant_result.insert(loc=restaurant_result.shape[1],column="Distance",value=resto_distance)
    restaurant_result = restaurant_result.sort_values(by=['Distance'])
    distance_list = restaurant_result.head(1).Distance.to_list()
    if(distance_list[0] > 0.5):
        restaurant_result = get_index_from_type(coords)
    #restaurant_index = restaurant_result.index.to_list()[0]
    return restaurant_result

In [84]:
res = get_index_from_type((22.141000,-100.923000))
res

,placeID,name,address,Rcuisine,latitude,longitude,rating,Distance
40,132869,Dominos Pizza,Ricardo B. Anaya,Pizzeria,22.141239,-100.923927,1.166667,0.099259
89,135054,Restaurante y Pescaderia Tampico,Ricardo B. Anaya 2700 Estrella de Oriente,Seafood,22.140627,-100.915657,1.000000,0.758624
33,132854,Sirlone,carr. mexico,International,22.137863,-100.938324,1.333333,1.618546
97,135071,Restaurante la Cantina,De La Estrella 2005 Estrella de Oriente,Bar,22.126375,-100.910927,1.000000,2.043016
96,135071,Restaurante la Cantina,De La Estrella 2005 Estrella de Oriente,Bar_Pub_Brewery,22.126375,-100.910927,1.000000,2.043016
100,135073,Restaurante Bar El Gallinero,Pascual M. Hernandez 210 Centro,Bar,22.147175,-100.974266,1.500000,5.332497
71,135032,Cafeteria y Restaurant El Pacifico,Constitucion 200 Centro,Contemporary,22.152481,-100.973488,1.178571,5.361027
47,132925,el pueblito,?,Mexican,22.153500,-100.976242,1.000000,5.663919
84,135052,La Cantina Restaurante,Ignacio Aldama 300 Centro,Bar_Pub_Brewery,22.150982,-100.977409,1.280000,5.720461
85,135052,La Cantina Restaurante,Ignacio Aldama 300 Centro,Bar,22.150982,-100.977409,1.280000,5.720461


In [46]:
def resto_search(name):
    return process.extractOne(name,restaurant_full_record['Rcuisine'])[0]

In [77]:
def main_function(longitude,latitude,resto_type=None):
    if(resto_type):
        resto_type = resto_search(resto_type)
    coords = (longitude, latitude)
    resto_index = get_index_from_type(coords,name=resto_type)
    index_arr = []
    for id in indices[resto_index.index.tolist()[0]]:
        index_arr.append(id)
    restaurant_result = restaurant_full_record.loc[index_arr]
    resto_distance = []
    for i in restaurant_result.index.tolist():
        resto_distance.append(distance.distance((restaurant_result.latitude[i],restaurant_result.longitude[i]),coords).km)
    restaurant_result.insert(loc=restaurant_result.shape[1],column="Distance",value=resto_distance)
    if(resto_type):
        restaurant_a_result = restaurant_result[restaurant_result['Rcuisine']==resto_type].reset_index(drop=True).sort_values(by="Distance")
        restaurant_b_result = restaurant_result[restaurant_result['Rcuisine']!=resto_type].reset_index(drop=True).sort_values(by="Distance")
        restaurant_result = pd.concat([restaurant_a_result,restaurant_b_result],ignore_index=True)
    else:
        restaurant_result = restaurant_result.sort_values(by='Distance',axis=0)
    return restaurant_result

In [114]:
index_arr = main_function(22.141000,-100.911000,"Seafood")
index_arr

,placeID,name,address,Rcuisine,latitude,longitude,rating,Distance
0,135054,Restaurante y Pescaderia Tampico,Ricardo B. Anaya 2700 Estrella de Oriente,Seafood,22.140627,-100.915657,1.000000,0.482193
1,135039,Restaurant de Mariscos de Picon,Miguel Barragan 46 Centro,Seafood,22.145893,-100.974869,1.250000,6.610815
2,135060,Restaurante Marisco Sam,Ignacio Allende 785 Centro,Seafood,22.156883,-100.978485,1.136364,7.180099
3,135075,Mariscos El Pescador,Himno Nacional 2104 Tangamanga,Seafood,22.139572,-100.991562,1.692308,8.312291
4,135049,Restaurante de Mariscos la Langosta,Cordillera de Los Alpes 675 Loma Verde,Seafood,22.135012,-101.028603,1.000000,12.150220
5,132869,Dominos Pizza,Ricardo B. Anaya,Pizzeria,22.141239,-100.923927,1.166667,1.333840
6,135071,Restaurante la Cantina,De La Estrella 2005 Estrella de Oriente,Bar,22.126375,-100.910927,1.000000,1.619452
7,135071,Restaurante la Cantina,De La Estrella 2005 Estrella de Oriente,Bar_Pub_Brewery,22.126375,-100.910927,1.000000,1.619452
8,132866,Chaires,Ricardo B. Anaya,Cafeteria,22.141220,-100.931313,1.400000,2.095579
9,132866,Chaires,Ricardo B. Anaya,Bakery,22.141220,-100.931313,1.400000,2.095579


In [116]:
print("Masukan Koordinat:")
print("latitude:")
latitude = input()
print("longitude:")
longitude = input()
print("gunakan filter jenis masakan (y/n)?")
inpt = input()
if(inpt == 'y'):
    print("masukan jenis makanan")
    jenis = input()
else:
    jenis = None;
print("Tunggu sistem rekomendasi berjalan...")
index_arr = main_function(float(latitude),float(longitude),jenis)
for i in index_arr.index.to_list():
    print("Rekomendasi Restoran")
    print("nama:"+index_arr.name.loc[i])
    print("alamat:"+index_arr.address.loc[i])
    print("jarak:"+str(index_arr.Distance.loc[i]))
    print("========================================")

Masukan Koordinat:
latitude:
22.141220	
longitude:
	-100.931313	
gunakan filter jenis masakan (y/n)?
n
Tunggu sistem rekomendasi berjalan...
Rekomendasi Restoran
nama:Chaires
alamat:Ricardo B. Anaya
jarak:4.643408558356005e-05
Rekomendasi Restoran
nama:Chaires
alamat:Ricardo B. Anaya
jarak:4.643408558356005e-05
Rekomendasi Restoran
nama:Tortas y hamburguesas el gordo
alamat:Ricardo B. Anaya
jarak:0.47638851362366424
Rekomendasi Restoran
nama:Cafeteria y Restaurant El Pacifico
alamat:Constitucion 200 Centro
jarak:4.525744438878581
Rekomendasi Restoran
nama:Luna Cafe
alamat:Francisco I. Madero 215
jarak:4.903679561908937
Rekomendasi Restoran
nama:Hamburguesas saul
alamat:Av. Saan Luis enttre moctezuma y salinas
jarak:5.08040746450587
Rekomendasi Restoran
nama:cafe punta del cielo
alamat:?
jarak:5.374492861723042
Rekomendasi Restoran
nama:Preambulo Wifi Zone Cafe
alamat:Anahuac 805
jarak:6.044623979183698
Rekomendasi Restoran
nama:Cafe Chaires
alamat:?
jarak:6.334262230444069
Rekomendasi 

In [71]:
resto = "Pizzeria"
coords = (22.141239,-100.923927)
resto_index = get_index_from_type(name=resto,coords=coords)
dasadass

In [ ]:
coords = (22.141239,-100.923927)
distance.distance((restaurant_full_record["latitude"][0],restaurant_full_record["longitude"]), coords).meters < 100